# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robertsa, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów. 
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from scipy import signal


# Obrazki
if not os.path.exists("jet.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/jet.png --no-check-certificate
if not os.path.exists("kw.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/kw.png --no-check-certificate
if not os.path.exists("moon.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/moon.png --no-check-certificate
if not os.path.exists("lenaSzum.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaSzum.png --no-check-certificate
if not os.path.exists("lena.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lena.png --no-check-certificate
if not os.path.exists("plansza.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/plansza.png --no-check-certificate


plansza = cv2.imread("plansza.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
def show(image, title = 0):
  plt.imshow(image)
  plt.gray()
  plt.xticks([]), plt.yticks([])
  if title != 0:
    plt.title(title)
  plt.show()


maska = np.ones((3,3))/9
new_plansza = cv2.filter2D(plansza, -1, maska)
diff = np.abs(plansza-new_plansza)

show(plansza,"obraz przed filtracją")

show(new_plansza,"obraz po filtracji")

show(diff,"Moduł z różnicy")




Największe różnice (kolor biały) występują  miejscu gwałtownej zmiany koloru. To znaczy, że nasz filtr wpływa głównie na te miejsca (zmiejszając przeskok kolorów - rozmazuje obraz)

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35. 
Wykorzystaj polecenie `plt.subplot`. 
Przeanalizuj wpływ rozmiaru maski na wynik. 

In [ ]:
def basic_filter (image, size):

    mask = np.ones((size,size))/(size**2)
    new_image = cv2.filter2D(image, -1, mask)
    return new_image


zad5, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,5, figsize=(13,5))

plansza3=basic_filter(plansza,3)
plansza5=basic_filter(plansza,5)
plansza9=basic_filter(plansza,9)
plansza15=basic_filter(plansza,15)
plansza35=basic_filter(plansza,35)

ax1.imshow(plansza3)
ax1.axis('off')
ax1.set_title("Okno o rozmiarze 3")

ax2.imshow(plansza5)
ax2.axis('off')
ax2.set_title("Okno o rozmiarze 5")


ax3.imshow(plansza9)
ax3.axis('off')
ax3.set_title("Okno o rozmiarze 39")

ax4.imshow(plansza15)
ax4.axis('off')
ax4.set_title("Okno o rozmiarze 15")

ax5.imshow(plansza35)
ax5.axis('off')
ax5.set_title("Okno o rozmiarze 35")


Im większy rozmiar maski w filtrze uśredniającym tym większe rozmazanie

6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
lena = cv2.imread("lena.png", cv2.IMREAD_GRAYSCALE)
new_lena = basic_filter(lena,3)

show(lena, "Orginalna Lena :D")
show(lena, "Lena po filtracji")
show(np.abs(lena-new_lena), "różnica Len")

7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości. 
Częściowo można go zniwelować poprzez odpowiedni dobór maski. 
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`. 
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
def Mfilter (image, mask):

    new_image = cv2.filter2D(image, -1, mask)
    return new_image


M = np.array([[1,2,1],[2,4,2],[1,2,1]])
M = M/np.sum(np.sum(M));

Mplansza=Mfilter(plansza,M)
MLena=Mfilter(lena,M)

diff = np.abs(plansza-Mplansza)
diff_Len = np.abs(lena-MLena)

fig, axs = plt.subplots(3, 2, figsize=(9,12))
axs[0,0].imshow(plansza)
axs[0,0].set_title("obraz przed filtracją")
axs[0,0].axis('off')

axs[0,1].imshow(lena)
axs[0,1].set_title("Lena przed filtracją")
axs[0,1].axis('off')


axs[1,0].imshow(Mplansza)
axs[1,0].set_title("obraz po filtracji")
axs[1,0].axis('off')

axs[1,1].imshow(MLena)
axs[1,1].set_title("Lena po filtracji")
axs[1,1].axis('off')


axs[2,0].imshow(diff)
axs[2,0].set_title("Moduł z różnicy")
axs[2,0].axis('off')

axs[2,1].imshow(diff_Len)
axs[2,1].set_title("Moduł z różnicy Len")
axs[2,1].axis('off')



8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa. 
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr `odchylenie standardowe` wpływa na `kształt` filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()
    

gaus = fgaussian(5,0.5)
mesh(gaus ,5)
gaus2 = fgaussian(5,0.2)
mesh(gaus2,5)
gaus3  = fgaussian(5,.9)
mesh(gaus3 ,5)


In [ ]:
lena_gaus_cv = cv2.GaussianBlur(lena,(3,3),0.4)
diff1=np.abs(lena-lena_gaus_cv)

gaus = fgaussian(5,0.4)
lena_gaus = Mfilter(lena, gaus)
diff2=np.abs(lena-lena_gaus_cv)

fig, axs = plt.subplots(2, 2, figsize=(12,12))

axs[0,0].imshow(lena_gaus_cv)
axs[0,0].set_title("Lena po filtracji CV2")
axs[0,0].axis('off')

axs[0,1].imshow(lena_gaus)
axs[0,1].set_title("Lena po filtracji")
axs[0,1].axis('off')


axs[1,0].imshow(diff1)
axs[1,0].set_title("Moduł z różnicy CV2")
axs[1,0].axis('off')

axs[1,1].imshow(diff2)
axs[1,1].set_title("Moduł z różnicy")
axs[1,1].axis('off')



### Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję `cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:
lena_szum = cv2.imread("lenaSzum.png", cv2.IMREAD_GRAYSCALE)
show(lena_szum)

lena_gaus2=cv2.GaussianBlur(lena_szum,(3,3),3)
diff1=np.abs(lena_szum-lena_gaus2)

lena_median=cv2.medianBlur(lena_szum,3)
diff2=np.abs(lena_szum-lena_median)

fig, axs = plt.subplots(2, 2, figsize=(12,12))

axs[0,0].imshow(lena_gaus2)
axs[0,0].set_title("Lena po filtracji gaussem")
axs[0,0].axis('off')

axs[0,1].imshow(lena_median)
axs[0,1].set_title("Lena po filtracji medianowej")
axs[0,1].axis('off')


axs[1,0].imshow(diff1)
axs[1,0].set_title("Moduł z różnicy - filt. gauss")
axs[1,0].axis('off')

axs[1,1].imshow(diff2)
axs[1,1].set_title("Moduł z różnicy - filt. medianowa")
axs[1,1].axis('off')

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

In [ ]:
lena_gaus=cv2.GaussianBlur(lena,(3,3),3)
diff1=np.abs(lena_szum-lena_gaus)

lena_median=cv2.medianBlur(lena,3)
diff2=np.abs(lena_szum-lena_median)



fig, axs = plt.subplots(3, 2, figsize=(9,15))
axs[0,0].imshow(lena)
axs[0,0].set_title("Lena przed filtracją gauss")
axs[0,0].axis('off')

axs[0,1].imshow(lena)
axs[0,1].set_title("Lena przed filtracją medianowej")
axs[0,1].axis('off')


axs[1,0].imshow(lena_gaus)
axs[1,0].set_title("Lena po filtracji gaus")
axs[1,0].axis('off')

axs[1,1].imshow(lena_median)
axs[1,1].set_title("Lena po filtracji medianowej")
axs[1,1].axis('off')


axs[2,0].imshow(diff1)
axs[2,0].set_title("Moduł z różnicy - filt. gauss")
axs[2,0].axis('off')

axs[2,1].imshow(diff2)
axs[2,1].set_title("Moduł z różnicy - filt. median")
axs[2,1].axis('off')

3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt `x`, `+`), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z `x` i mediana z `+`,
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
lena_10 = cv2.imread("lena.png", cv2.IMREAD_GRAYSCALE)

for i in range (10):
  lena_10=cv2.medianBlur(lena_10,5)

diff10=np.abs(lena_10-lena)

show(lena_10, "Lena - postreryzacja")
show(diff10, 'różncia Len')

## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M = 
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\tag{1}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez sumę wag dodatnich (ewentualnie sumę modułów wszystkich wag).
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodanie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje. 
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach. 

In [ ]:
moon = cv2.imread("moon.png", cv2.IMREAD_GRAYSCALE)

M=np.array([[0,1,0],[1, -4, 1], [0,1,0]])
M = M/np.sum(np.sum(np.abs(M)));

moon_lap = Mfilter(moon, M)

moon_lap_abs = (np.abs(moon_lap)/ np.max( np.abs(moon_lap)) * 255).astype('uint8')
moon_lap_128 = moon_lap + 128

diff1=np.abs(moon_lap_abs-moon)
diff2=np.abs(moon_lap_128-moon)


fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(13,5))

ax1.imshow(moon)
ax1.axis('off')
ax1.set_title("Obraz orginalny")

ax2.imshow(moon_lap_abs)
ax2.axis('off')
ax2.set_title("Wartość bezwzględna")


ax3.imshow(moon_lap_128)
ax3.axis('off')
ax3.set_title("+128")

7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:

fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(16,7))


diff = cv2.subtract(moon.astype('int16'), moon_lap_128.astype('int16'))
diff = np.abs(diff)/2
diff = diff.astype('uint8')

summ= cv2.add(moon.astype('uint16'), moon_lap_128.astype('uint16'))
summ = (summ/2).astype('uint8')

ax1.imshow(moon)
ax1.axis('off')
ax1.set_title("obraz orginalny")

ax2.imshow(diff)
ax2.axis('off')
ax2.set_title("Różnica")


ax3.imshow(summ)
ax3.axis('off')
ax3.set_title("suma")


In [ ]:

diff = cv2.subtract(moon.astype('int16'), moon_lap_abs.astype('int16'))
diff = np.abs(diff)
diff = diff.astype('uint8')

summ= cv2.add(moon.astype('uint16'), moon_lap_abs.astype('uint16'))
summ = (summ/2).astype('uint8')


fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(16,7))


ax1.imshow(moon, vmin=0, vmax=256)
ax1.axis('off')
ax1.set_title("obraz orginalny")

ax2.imshow(diff, vmin=0, vmax=256)
ax2.axis('off')
ax2.set_title("Różnica")


ax3.imshow(summ, vmin=0, vmax=256)
ax3.axis('off')
ax3.set_title("suma")

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\tag{2}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\tag{3}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie `prostopadłe`):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\tag{4}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela. 

In [ ]:
moon = cv2.imread("kw.png", cv2.IMREAD_GRAYSCALE)

R1 = [[0,0,0],[-1,0,0],[0,1,0]]
R1 = R1/np.sum(np.sum(np.abs(R1)))


moon_lap = Mfilter(moon, R1)

moon_lap_abs = (np.abs(moon_lap)/ np.max( np.abs(moon_lap)) * 255).astype('uint8')
moon_lap_128 = moon_lap
norm = np.zeros(moon_lap_128.shape[0], 'uint8')
moon_lap_128 = cv2.normalize(moon_lap_128, norm, 0, 255,cv2.NORM_MINMAX)
diff1=np.abs(moon_lap_abs-moon)
diff2=np.abs(moon_lap_128-moon)


fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(13,5))

ax1.imshow(moon)
ax1.axis('off')
ax1.set_title("Obraz orginalny")

ax2.imshow(moon_lap_abs)
ax2.axis('off')
ax2.set_title("Wartość bezwzględna")


ax3.imshow(moon_lap_128)
ax3.axis('off')
ax3.set_title("+128")

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(16,7))


diff = cv2.subtract(moon.astype('int16'), moon_lap_128.astype('int16'))
diff = np.abs(diff)/2
diff = diff.astype('uint8')

summ= cv2.add(moon.astype('uint16'), moon_lap_128.astype('uint16'))
summ = (summ/2).astype('uint8')

ax1.imshow(moon)
ax1.axis('off')
ax1.set_title("obraz orginalny")

ax2.imshow(diff)
ax2.axis('off')
ax2.set_title("Różnica")


ax3.imshow(summ)
ax3.axis('off')
ax3.set_title("suma")

In [ ]:

diff = cv2.subtract(moon.astype('int16'), moon_lap_abs.astype('int16'))
diff = np.abs(diff)
diff = diff.astype('uint8')

summ= cv2.add(moon.astype('uint16'), moon_lap_abs.astype('uint16'))
summ = (summ/2).astype('uint8')


fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(16,7))


ax1.imshow(moon)
ax1.axis('off')
ax1.set_title("obraz orginalny")

ax2.imshow(diff)
ax2.axis('off')
ax2.set_title("Różnica")


ax3.imshow(summ)
ax3.axis('off')
ax3.set_title("suma")

In [ ]:
def fun(moon, R1):
    
  moon_lap = Mfilter(moon, R1)

  moon_lap_abs = (np.abs(moon_lap)/ np.max( np.abs(moon_lap)) * 255).astype('uint8')
  moon_lap_128 = moon_lap + 128

  fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(13,5))

  ax1.imshow(moon)
  ax1.axis('off')
  ax1.set_title("Obraz orginalny")

  ax2.imshow(moon_lap_abs)
  ax2.axis('off')
  ax2.set_title("Wartość bezwzględna")


  ax3.imshow(moon_lap_128)
  ax3.axis('off')
  ax3.set_title("+128")

  fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(16,7))


  diff = cv2.subtract(moon.astype('int16'), moon_lap_128.astype('int16'))
  diff = np.abs(diff)/2
  diff = diff.astype('uint8')

  summ= cv2.add(moon.astype('uint16'), moon_lap_128.astype('uint16'))
  summ = (summ/2).astype('uint8')

  ax1.imshow(moon)
  ax1.axis('off')
  ax1.set_title("obraz orginalny")

  ax2.imshow(diff)
  ax2.axis('off')
  ax2.set_title("Różnica")


  ax3.imshow(summ)
  ax3.axis('off')
  ax3.set_title("suma")

  
  diff = cv2.subtract(moon.astype('int16'), moon_lap_abs.astype('int16'))
  diff = np.abs(diff)
  diff = diff.astype('uint8')

  summ= cv2.add(moon.astype('uint16'), moon_lap_abs.astype('uint16'))
  summ = (summ/2).astype('uint8')


  fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(16,7))


  ax1.imshow(moon)
  ax1.axis('off')
  ax1.set_title("obraz orginalny")

  ax2.imshow(diff)
  ax2.axis('off')
  ax2.set_title("Różnica")


  ax3.imshow(summ)
  ax3.axis('off')



In [ ]:
R2 = [[0,0,0],[0,0,-1],[0,1,0]]
R2 = R2/np.sum(np.sum(np.abs(R2)))

P1 = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
P1 = P1/np.sum(np.sum(np.abs(P1)))

P2 = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
P2 = P2/np.sum(np.sum(np.abs(P2)))


S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
S1 = S1/np.sum(np.sum(np.abs(S1)))

S2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
S2 = S2/np.sum(np.sum(np.abs(S2)))

fun(moon, R2)


In [ ]:
fun(moon, R2)

In [ ]:
fun(moon, P1)

In [ ]:
fun(moon, P2)

In [ ]:
fun(moon, S1)

In [ ]:
fun(moon, S2)

5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\tag{5}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
kw = cv2.imread("kw.png", cv2.IMREAD_GRAYSCALE)
kw=kw.astype('int16')

I1=Mfilter(kw, S1)
I2=Mfilter(kw, S2)

I1=np.power(I1,2)
I2=np.power(I2,2)

OW=np.sqrt(I1 + I2).astype('uint8')

show(OW)


6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia). 
Zaimplementuj tę wersję. 

In [ ]:
I1=Mfilter(kw, S1)
I2=Mfilter(kw, S2)

OW=((np.abs(I1) + np.abs(I2))/2).astype('uint8')

show(OW)

7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:
jet = cv2.imread("jet.png", cv2.IMREAD_GRAYSCALE)
jet=jet.astype('int16')

I1=Mfilter(jet, S1)
I2=Mfilter(jet, S2)

I1=np.power(I1,2)
I2=np.power(I2,2)

OW=np.sqrt(I1 + I2).astype('uint8')

show(OW)

In [ ]:
I1=Mfilter(jet, S1)
I2=Mfilter(jet, S2)

OW=((np.abs(I1) + np.abs(I2))/2).astype('uint8')

show(OW)